# Building a Spam Filter with Naive Bayes

---

## 1. Introduction

In this project, we will **use the multinomial Naive Bayes algorithm to create a spam filter** using a [dataset of SMS messages](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The program will:

- Learn how humans classify messages.
- Use that knowledge to estimate probabilities of whether a new message is spam or non-spam.
- Classify a new message based on these probability values:
    - If the probability for spam is greater, then it classifies the message as spam. 
    - Otherwise, it classifies it as non-spam.
    - If the two probability values are equal, then we may need a human to classify the message.

Our aim is to achieve an **accuracy of at least 80%** i.e. at least 80% of the new messages will be classified correctly as spam or non-spam.

---

## 2. Open and Explore the Data

Let's start by reading the dataset of messages. 

**Note that due to the nature of spam messages, the data contains content that may be offensive to some users.**

In [1]:
import pandas as pd

# Csv file has tab separator, no header row
sms_spam = pd.read_csv('SMSSpamCollection', sep = '\t', header = None, names = ['Label', 'SMS'])

# Number of rows and columns
print(f'The dataset has {sms_spam.shape[0]} rows and {sms_spam.shape[1]} columns.')

# Display first 5 rows
sms_spam.head()

There are 5572 messages in the dataset, and each is labelled as `spam` or `ham` (i.e. non-spam) by human input.

We can also find out the percentage of spam messages in the data.

In [2]:
print('Type:   Percentage (%)')
print(sms_spam['Label'].value_counts(normalize = True) * 100)

Type:   Percentage (%)
ham     86.593683
spam    13.406317
Name: Label, dtype: float64


About 87% of the messages in the dataset are non-spam, while the remaining 13% are spam.

---

## 3. Segregate Training and Test Data

To facilitate testing of the spam filter, let's split the data into two categories:
- a training set to train the program on how to classify messages and
- a test set to verify the performance of the program in classifying new messages.

We will use 80% of the messages for training and the remaining 20% for testing.

In [3]:
# Randomize order of messages in dataset
sms_random = sms_spam.sample(frac = 1, random_state = 1)

# Split randomized dataset into training and test sets
train_index = round(len(sms_random) * 0.8)
sms_train = sms_random.iloc[:train_index].reset_index(drop = True)
sms_test = sms_random.iloc[train_index:].reset_index(drop = True)

# Verify size of training and test sets
print(f'There are {len(sms_train)} messages in the training set.')
print(f'There are {len(sms_test)} messages in the test set.')

print('\n')

# Show percentage of spam and ham in both sets
print('Type:   Percentage (%) in Training Set')
print(sms_train['Label'].value_counts(normalize = True) * 100)

print('\n')

print('Type:   Percentage (%) in Test Set')
print(sms_test['Label'].value_counts(normalize = True) * 100)

There are 4458 messages in the training set.
There are 1114 messages in the test set.


Type:   Percentage (%) in Training Set
ham     86.54105
spam    13.45895
Name: Label, dtype: float64


Type:   Percentage (%) in Test Set
ham     86.804309
spam    13.195691
Name: Label, dtype: float64


The proportions of spam and ham messages in the training and test sets are similar to that in the full dataset.


---

## 4. Clean and Transform the Data

We will transform the training data into a format that is more convenient to train the algorithm, by replacing the `SMS` column with a series of new columns. 

Each new column will represent a unique word in lower case, with the value indicating the number of times the word occurs in each message. This means that we will consider `secret` and `SECRET` as the same word.

We will clean the `SMS` column with the following steps:
- remove all punctuation,
- convert all words to lower case and
- split the message into a list of words.

In [4]:
# Remove all punctuation
sms_train['SMS'] = sms_train['SMS'].str.replace('\W', ' ')

# Convert words to lower case
sms_train['SMS'] = sms_train['SMS'].str.lower()

# Split string at space character
sms_train['SMS'] = sms_train['SMS'].str.split()

# Verify first few rows
sms_train.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


Next, we will create a `vocabulary` list which contains all the unique words from the messages in the training data.

In [5]:
# Initiate empty list
vocabulary = []

# Iterate over the SMS column
for sms in sms_train['SMS']:

    # Append each string into the list
    for word in sms:
        vocabulary.append(word)

# Transform into a set to remove duplicates
vocabulary = set(vocabulary)

print(f'There are {len(vocabulary)} unique words in the vocabulary.')

There are 7783 unique words in the vocabulary.


We will use the vocabulary to build a dictionary named `word_counts_per_sms` where:
- each key corresponds to a unique word from the vocabulary and
- each value is a list of the word counts in each message from the training set.

In [6]:
# Initialize dictionary of unique words with zero values
word_counts_per_sms = {unique_word: [0] * len(sms_train['SMS']) 
                       for unique_word in vocabulary}

# Compute counts for each word in each message
for index, sms in enumerate(sms_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

# Convert dictionary to dataframe
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,although,got,lovin,imposter,burn,bear,600,515,ahmad,janinexx,...,14tcr,03,emigrated,2morrowxxxx,downloaded,fakeye,eerie,pobox75ldns7,09061221061,str
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We can concatenate this new dataframe with the original training data, to retain the `Label` and `SMS` columns.

In [7]:
# Concatenate word_counts with training data
sms_train_clean = pd.concat([sms_train, word_counts], axis = 1)
sms_train_clean.head()

,Label,SMS,although,got,lovin,imposter,burn,bear,600,515,...,14tcr,03,emigrated,2morrowxxxx,downloaded,fakeye,eerie,pobox75ldns7,09061221061,str
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

## 5. Build the Spam Filter

We will begin to create the spam filter with the cleaned training data, by defining the following constant terms:
- `p_spam`: probability of a message being spam
- `p_ham`: probability of a message being non-spam
- `n_spam`: number of words in all the spam messages
- `n_ham`: number of words in all the non-spam messages
- `n_vocabulary`: number of words in the vocabulary
- `alpha`: Laplace smoothing parameter

In [8]:
# Calculate probabilities
[p_ham, p_spam] = sms_train_clean['Label'].value_counts(normalize = True)

# Isolate spam and ham messages
sms_train_spam = sms_train_clean[sms_train_clean['Label'] == 'spam']
sms_train_ham = sms_train_clean[sms_train_clean['Label'] == 'ham']

# Calculate number of words in messages
n_spam = sum(sms_train_spam['SMS'].apply(len))
n_ham = sum(sms_train_ham['SMS'].apply(len))

# Define other variables
n_vocabulary = len(vocabulary)
alpha = 1

# Display and verify all variables
print(f'p_spam = {p_spam:.5f}')
print(f'p_ham = {p_ham:.5f}')
print(f'n_spam = {n_spam}')
print(f'n_ham = {n_ham}')
print(f'n_vocabulary = {n_vocabulary}')
print(f'alpha = {alpha}')

p_spam = 0.13459
p_ham = 0.86541
n_spam = 15190
n_ham = 57237
n_vocabulary = 7783
alpha = 1


To implement the Naive Bayes algorithm, we also require a few other parameters:
- P(word|Spam): the posterior probability that a spam message contains a certain word
- P(word|Ham): the posterior probability that a non-spam message contains a certain word

We will compute the values of these parameters for every word in the vocabulary, then store them in two dictionaries (`spam_parameters` and `ham_parameters`).

In [9]:
# Initiate dictionaries with zeros as parameters
spam_parameters = {unique_word: 0 for unique_word in vocabulary}
ham_parameters = {unique_word: 0 for unique_word in vocabulary}

# Calculate posterior probabilities for each word in vocabulary
for unique_word in vocabulary:
    n_word_given_spam = sms_train_spam[unique_word].sum()
    spam_parameters[unique_word] = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocabulary)

    n_word_given_ham = sms_train_ham[unique_word].sum()
    ham_parameters[unique_word] = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    
# Display sample values from each dictionary
print('Sample data from spam_parameters:')
print(list(spam_parameters.items())[:5])
print('Sample data from ham_parameters:')
print(list(ham_parameters.items())[:5])

Sample data from spam_parameters:
[('although', 4.3529360553693465e-05), ('got', 0.00021764680276846734), ('lovin', 4.3529360553693465e-05), ('imposter', 4.3529360553693465e-05), ('burn', 4.3529360553693465e-05)]
Sample data from ham_parameters:
[('although', 4.6139649338665025e-05), ('got', 0.002660719778529683), ('lovin', 3.075976622577668e-05), ('imposter', 3.075976622577668e-05), ('burn', 3.075976622577668e-05)]


Let's create the spam filter using the constants and parameters above. We will define the spam filter as a function that:
- receives a new message as a string input,
- calculates `P(Spam|message)` - posterior probability that the message is spam,
- calculates `P(Ham|message)` - posterior probability that the message is non-spam and 
- classify the message as spam or non-spam.

The new messages may contain words that are not part of the training vocabulary. Hence, we will ignore these new words in the probability computation.

We will define the function below and verify its functionality on two messages.

In [10]:
import re

# Spam filter function with string input
def classify(message):
    
    # Remove punctuation from string
    message = re.sub('\W', ' ', message)
    # Convert string to lower case
    message = message.lower()
    # Split string into list of words
    message = message.split()
    
    # Initialize p_spam_given_message and p_ham_given_message values
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Iterate over each word in message
    for word in message:
        
        # Update p_spam_given_message if the word is in spam_parameters
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]

        # Update p_ham_given_message if the word is in ham_parameters 
        if word in ham_parameters:
            p_ham_given_message *= ham_parameters[word]
                
    # Display probabilities of spam and ham
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    # Classify message
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities, have a human classify this!')
        
# Test message 1
print('WINNER!! This is the secret code to unlock the money: C3421.')
classify('WINNER!! This is the secret code to unlock the money: C3421.')

print('\n')

# Test message 2
print('Sounds good, Tom, then see u there')
classify('Sounds good, Tom, then see u there')

WINNER!! This is the secret code to unlock the money: C3421.
P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


Sounds good, Tom, then see u there
P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


---

## 6. Test the Spam Filter

Let's determine how well the spam filter performs on the test set. 

For every message in the test set, we will use the algorithm to generate a classification label and compare this against the actual label.

In [11]:
import re

# Spam filter function with string input
def classify_test_set(message):
    
    # Remove punctuation from string
    message = re.sub('\W', ' ', message)
    # Convert string to lower case
    message = message.lower()
    # Split string into list of words
    message = message.split()
    
    # Initialize p_spam_given_message and p_ham_given_message values
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Iterate over each word in message
    for word in message:
        
        # Update p_spam_given_message if the word is in spam_parameters
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]

        # Update p_ham_given_message if the word is in ham_parameters 
        if word in ham_parameters:
            p_ham_given_message *= ham_parameters[word]
                  
    # Classify message      
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    
# Compare actual and predicted labels
sms_test['predicted'] = sms_test['SMS'].apply(classify_test_set)
sms_test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


We can now calculate the spam filter accuracy as the percentage of test set messages that are correctly classified.

In [12]:
# Initialize variable for number of correct messages
correct = 0

# Total number of messages in the test set
total = len(sms_test)

for row in sms_test.iterrows():
    row = row[1]
    
    # Messages classified correctly
    if row['Label'] == row['predicted']:
        correct += 1

accuracy = correct / total * 100
print(f'{correct} out of {total} messages were classified correctly.')
print(f'The accuracy of the spam filter is {accuracy:.2f}%.')

1100 out of 1114 messages were classified correctly.
The accuracy of the spam filter is 98.74%.


---

## 7. Conclusion

- Using the Naive Bayes algorithm, we built a spam filter for messages.
- It achieved an accuracy of 98.74% on the test data, which exceeded the original target of 80%.